Criar script para ler os datasets em /exercicios/municipios-estados — o script deve:
criar uma pasta nomeada com a sigla da UF para cada estado encontrado no arquivo JSON;
organizar as cidades por estado um único arquivo CSV, nomeado como cidades.csv;
salvar esse arquivo cidades.csv dentro da pasta da UF (estado) correspondente.
Importar as pastas e arquivos salvos para um bucket no MinIO.
Exportar os dados para o MongoDB e visualizá-los a partir do banco de dados.  


In [1]:
!pip install pandas
!pip install pymongo
!pip install boto3

In [2]:
import numpy as np
import pandas as pd
from io import BytesIO
from pymongo import MongoClient
import boto3
from io import StringIO 

In [3]:
client = boto3.client('s3', 
    endpoint_url='http://awari-minio-nginx:9000',
    aws_access_key_id="ZxgL75QlD7LYmzqU",
    aws_secret_access_key='Q4yxRqfMNVLosGslPksramjS38axzdsX',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name='sa-east-1'
)

In [11]:
df_estados = pd.read_csv('./arquivos/estados.csv')

In [12]:
df_estados

,codigo_uf,uf,nome,latitude,longitude,regiao
0,11,RO,Rondônia,-10.83,-63.34,Norte
1,12,AC,Acre,-8.77,-70.55,Norte
2,13,AM,Amazonas,-3.47,-65.10,Norte
3,14,RR,Roraima,1.99,-61.33,Norte
4,15,PA,Pará,-3.79,-52.48,Norte
5,16,AP,Amapá,1.41,-51.77,Norte
6,17,TO,Tocantins,-9.46,-48.26,Norte
7,21,MA,Maranhão,-5.42,-45.44,Nordeste
8,22,PI,Piauí,-6.60,-42.28,Nordeste
9,23,CE,Ceará,-5.20,-39.53,Nordeste


In [8]:
df_municipios = pd.read_csv('./arquivos/municipios.csv')

In [10]:
df_municipios

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo
...,...,...,...,...,...,...,...,...,...
5565,2933604,Xique-Xique,-10.82300,-42.7245,0,29,3971,74,America/Sao_Paulo
5566,2517407,Zabelê,-8.07901,-37.1057,0,25,542,83,America/Sao_Paulo
5567,3557154,Zacarias,-21.05060,-50.0552,0,35,2973,18,America/Sao_Paulo
5568,2114007,Zé Doca,-3.27014,-45.6553,0,21,1287,98,America/Sao_Paulo


In [16]:
#df_municipios.join(df_estados[[ 'codigo_uf', 'uf']], on='codigo_uf', lsuffix='_mun', rsuffix='_est')

In [17]:
df_completo = df_municipios.merge(df_estados[[ 'codigo_uf', 'uf']], on='codigo_uf')

In [21]:
df_completo.head()


,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario,uf
0,5200050,Abadia de Goiás,-16.7573,-49.4412,0,52,1050,62,America/Sao_Paulo,GO
1,5200100,Abadiânia,-16.1970,-48.7057,0,52,9201,62,America/Sao_Paulo,GO
2,5200134,Acreúna,-17.3960,-50.3749,0,52,9645,64,America/Sao_Paulo,GO
3,5200159,Adelândia,-16.4127,-50.1657,0,52,9769,64,America/Sao_Paulo,GO
4,5200175,Água Fria de Goiás,-14.9778,-47.7823,0,52,9771,62,America/Sao_Paulo,GO


In [24]:
# 1. Filtrar por UF
# 2. para cada dataframe filtrado
# 3. enviar para o minio/s3 para a pasta com o nome da UF

for uf in df_completo['uf'].unique():
    df_tmp = df_completo.loc[ df_completo['uf'] == uf ]
    client.put_object(Body=df_tmp.to_csv(), Bucket='aula-06', Key="{}/cidades.csv".format(uf)) ou #f'{uf}/cidades.csv'

    